In [6]:
import os
import time
import random
import time
import requests 
import re
import logging
import subprocess
from subprocess import Popen
from sys import platform
import os, sys
import logging
import json
import threading
import numpy as np

import subprocess
import time
import signal
from typing import Dict
import pandas as pd
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher


from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest
from netunicorn.library.tasks.measurements.ping import Ping
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher
from netunicorn.base import Result, Failure, Success, Task, TaskDispatcher
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.library.tasks.video_watchers.youtube_watcher import WatchYouTubeVideo
from netunicorn.library.tasks.video_watchers.vimeo_watcher import WatchVimeoVideo
from netunicorn.library.tasks.video_watchers.twitch_watcher import WatchTwitchStream

from typing import Dict
from typing import Optional
from enum import IntEnum
from datetime import datetime
import matplotlib.pyplot as plt
from returns.pipeline import is_successful
from returns.result import Failure

In [2]:
class ReadCaptureFile(Task):
    def __init__(self, filepath: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filepath = filepath

    def run(self):
        try:
            # Read the capture file contents
            with open(self.filepath, 'rb') as file:
                data = file.read()
                # For now, just return the raw bytes (you could later parse it)
                return Success(data)
        except Exception as e:
            return Failure(f"Error reading capture file: {e}")


In [3]:
def execute_pipeline(pipeline, working_node, experiment_label):
    client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
    nodes = client.get_nodes()
    working_nodes = nodes.filter(lambda node: node.name.startswith(working_node)).take(1)
    experiment = Experiment().map(pipeline, working_nodes)

    try:
        client.delete_experiment(experiment_label)
    except RemoteClientException:
        pass

    # Prepare Experiment
    client.prepare_experiment(experiment, experiment_label)
    while True:
        info = client.get_experiment_status(experiment_label)
        print(info.status)
        if info.status == ExperimentStatus.READY:
            break
        time.sleep(20)

    time.sleep(5)

    # Execute Experiment
    client.start_execution(experiment_label)
    while True:
        info = client.get_experiment_status(experiment_label)
        print(info.status)
        if info.status != ExperimentStatus.RUNNING:
            break
        time.sleep(20)
    return info

In [4]:
def display_results(info):
    # Get Results
    for report in info.execution_result:
        print(f"Node name: {report.node.name}")
        print(f"Error: {report.error}")

        if report.result is None:
            print("report.result is EMPTY..")
            continue

        result, log = report.result  # report stores results of execution and corresponding log

        # result is a returns.result.Result object, could be Success of Failure
        print(f"Result is: {type(result)}")
        if is_successful(result):
            data = result.unwrap()
        else:
            data = result.failure()
        try:
            for key, value in data.items():
                print(f"{key}: {value}")
        except:
            print(f"No attribute 'items' in result")

        # we also can explore logs
        for line in log:
            print(line.strip())
        print()

In [8]:
# TODO: fill in your login / password below
netunicorn_login = 'cs190n2'
netunicorn_password = 'eVmBjYmp'

NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', netunicorn_login)
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', netunicorn_password)

client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)

Health Check: True
[<Uncountable node pool with next node template: [aws-fargate-A-cs190n2-, aws-fargate-B-cs190n2-, aws-fargate-ARM64-cs190n2-]>]


In [9]:
# TODO: fill in the name of your working node
working_node = 'aws-fargate-A-cs190n2-'

In [10]:
data = pd.read_csv('Data/ssim_2019-01-26T11_2019-01-27T11.csv') 

In [15]:
video_sent = pd.read_csv('Data/video_sent_2019-01-26T11_2019-01-27T11.csv')
video_acked = pd.read_csv('Data/video_acked_2019-01-26T11_2019-01-27T11.csv')
expt_settings = pd.read_csv('Data/expt_settings', sep='\t')

In [16]:
# Merge on common columns to align each chunk sent with its ack

merged_data = pd.merge(
    video_sent, 
    video_acked, 
    on=['session_id', 'index', 'video_ts'], 
    suffixes=('_sent', '_acked')
)

# Display first few rows of the merged data to verify
print(merged_data.head())

    time (ns GMT)_sent                                    session_id  index  \
0  1548500400788000000  kaXlFjXCNjAgx4nsvYsDetENDzv04n/e7R4HYutFWt0=      0   
1  1548500401993000000  j1E2fJfiOe5TU3Fj3lC1cgA4CsKjg2NJYBcxdVcdJfk=      0   
2  1548500402853000000  kaXlFjXCNjAgx4nsvYsDetENDzv04n/e7R4HYutFWt0=      0   
3  1548500402957000000  j1E2fJfiOe5TU3Fj3lC1cgA4CsKjg2NJYBcxdVcdJfk=      0   
4  1548500404848000000  kaXlFjXCNjAgx4nsvYsDetENDzv04n/e7R4HYutFWt0=      0   

   expt_id_sent channel_sent     video_ts       format    size  ssim_index  \
0           232          abc  38279961720  1280x720-20  579932    0.981149   
1           237          abc  38280322080  1280x720-20  269778    0.986832   
2           232          abc  38280141900  1280x720-20  475807    0.983754   
3           237          abc  38280502260  1280x720-20  322657    0.986481   
4           232          abc  38280322080  1280x720-20  269778    0.986832   

   cwnd  in_flight  min_rtt     rtt  delivery_rate  time

In [17]:

merged_data.rename(columns={'time (ns GMT)_sent': 'time_sent', 'time (ns GMT)_acked': 'time_acked'}, inplace=True)

merged_data['download_time'] = (merged_data['size'] - merged_data['delivery_rate']) / 1e9  # Convert nanoseconds to seconds
print(merged_data[['session_id', 'index', 'download_time']].head())


                                     session_id  index  download_time
0  kaXlFjXCNjAgx4nsvYsDetENDzv04n/e7R4HYutFWt0=      0      -0.000090
1  j1E2fJfiOe5TU3Fj3lC1cgA4CsKjg2NJYBcxdVcdJfk=      0      -0.010639
2  kaXlFjXCNjAgx4nsvYsDetENDzv04n/e7R4HYutFWt0=      0      -0.002424
3  j1E2fJfiOe5TU3Fj3lC1cgA4CsKjg2NJYBcxdVcdJfk=      0      -0.010586
4  kaXlFjXCNjAgx4nsvYsDetENDzv04n/e7R4HYutFWt0=      0      -0.003570


In [11]:
import signal
import subprocess
import time
from typing import List, Optional

from netunicorn.base import (
    Architecture,
    Failure,
    Node,
    Result,
    Success,
    Task,
    TaskDispatcher,
)
from netunicorn.library.tasks.tasks_utils import subprocess_run


class StartCapture(TaskDispatcher):
    def __init__(
        self, filepath: str, arguments: Optional[List[str]] = None, *args, **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.filepath = filepath
        self.arguments = arguments

        self.linux_implementation = StartCaptureLinuxImplementation(
            filepath=self.filepath, arguments=self.arguments, *args, **kwargs
        )

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation

        raise NotImplementedError(
            f"StartCapture is not implemented for {node.architecture}"
        )


class StartCaptureLinuxImplementation(Task):
    requirements = ["sudo apt-get update", "sudo apt-get install -y tcpdump"]

    def __init__(
        self, filepath: str, arguments: Optional[List[str]] = None, *args, **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.arguments = arguments or []
        self.filepath = filepath

    def run(self) -> Result:
        signal.signal(signal.SIGCHLD, signal.SIG_IGN)
        filter_expression = "(host 8.8.8.8 or host 1.0.0.1)"
        # Start tcpdump with options to capture packet size
        proc = subprocess.Popen(
            [
                "tcpdump",
                "-n",  # Don't resolve hostnames
                "-e",  # Display link-layer information, including packet size
                "-vv",  # Extra verbosity to show packet details
                "-U",  # Unbuffered output
                "-w", self.filepath,
                filter_expression,
            ] + self.arguments,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        time.sleep(2)
        if (exit_code := proc.poll()) is None:  # not finished yet
            return Success(proc.pid)

        text = ""
        if proc.stdout:
            text += proc.stdout.read().decode("utf-8") + "\n"
        if proc.stderr:
            text += proc.stderr.read().decode("utf-8")
        return Failure(f"Tcpdump terminated with return code {exit_code}" + text)


class StopNamedCapture(TaskDispatcher):
    def __init__(self, start_capture_task_name: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.start_capture_task_name = start_capture_task_name
        self.linux_implementation = StopNamedCaptureLinuxImplementation(
            capture_task_name=self.start_capture_task_name,
            *args,
            **kwargs,
        )

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation

        raise NotImplementedError(
            f"StopCapture is not implemented for {node.architecture}"
        )


class StopNamedCaptureLinuxImplementation(Task):
    requirements = [
        "sudo apt-get update",
        "sudo apt-get install -y tcpdump",
        "sudo apt-get install -y procps",
    ]

    def __init__(self, capture_task_name: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.capture_task_name = capture_task_name

    def run(self):
        signal.signal(signal.SIGCHLD, signal.SIG_IGN)
        pid = self.previous_steps.get(
            self.capture_task_name, [Failure("Named StartCapture not found")]
        )[-1]
        if isinstance(pid, Failure):
            return pid

        pid = pid.unwrap()
        return subprocess_run(["kill", str(pid)])


In [12]:
class DisplayCaptureFile(Task):
    def __init__(self, filepath: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filepath = filepath

    def run(self) -> Result:
        try:
            # Run tcpdump to read the pcap file and display packet details
            proc = subprocess.Popen(
                ["tcpdump", "-nn", "-vv", "-e", "-r", self.filepath],  # -e shows link-layer information (size included)
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
            )
            stdout, stderr = proc.communicate()
            
            if proc.returncode == 0:
                # Print the tcpdump output (which includes packet details like size)
                return Success(stdout.decode("utf-8"))
            
            # If there is an error, capture and return the stderr
            return Failure(stderr.decode("utf-8"))
        
        except Exception as e:
            # If there's an exception during the process, return the error message
            return Failure(str(e))


In [ ]:
"""
Selenium-based Vimeo watcher
"""
import os
import random
import subprocess
import time
from typing import Optional

from netunicorn.base import Failure, Result, Success, Task, TaskDispatcher
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node


def watch(
    url: str, duration: Optional[int] = 100, chrome_location: Optional[str] = None, webdriver_arguments: Optional[list] = None
) -> Result[str, str]:
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service

    display_number = random.randint(100, 500)
    xvfb_process = subprocess.Popen(
        ["Xvfb", f":{display_number}", "-screen", "0", "1920x1080x24"]
    )
    os.environ["DISPLAY"] = f":{display_number}"

    options = Options()
    options.add_argument("--no-sandbox")
    options.add_argument("--autoplay-policy=no-user-gesture-required")
    options.add_argument("--disable-dev-shm-usage")
    if webdriver_arguments:
        for argument in webdriver_arguments:
            options.add_argument(argument)
    if chrome_location:
        options.binary_location = chrome_location
    driver = webdriver.Chrome(service=Service(), options=options)
    time.sleep(1)
    driver.get(url)

    paused = driver.execute_script(
        "return document.getElementsByTagName('video')[0].paused"
    )
    if paused is None:
        driver.close()
        xvfb_process.kill()
        return Failure("Failed to get video status")

    if paused:
        driver.execute_script("document.getElementsByTagName('video')[0].play()")
        time.sleep(2)
        paused = driver.execute_script(
            "return document.getElementsByTagName('video')[0].paused"
        )
        if paused:
            driver.close()
            xvfb_process.kill()
            return Failure("Couldn't start the video: unknown error")

    if duration:
        time.sleep(duration)
        result = Success(f"Video finished by timeout: {duration} seconds")
    else:
        paused = driver.execute_script(
            "return document.getElementsByTagName('video')[0].paused"
        )
        while not paused:
            time.sleep(2)
            paused = driver.execute_script(
                "return document.getElementsByTagName('video')[0].paused"
            )
        result = Success("Video finished by reaching the end")

    driver.close()
    xvfb_process.kill()
    return result


class WatchVimeoVideo(TaskDispatcher):
    def __init__(
            self,
            video_url: str,
            duration: Optional[int] = None,
            chrome_location: Optional[str] = None,
            webdriver_arguments: Optional[list] = None,
            *args, **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.video_url = video_url
        self.duration = duration
        self.chrome_location = chrome_location
        self.webdriver_arguments = webdriver_arguments
        self.linux_implementation = WatchVimeoVideoLinuxImplementation(
            self.video_url,
            self.duration,
            self.chrome_location,
            self.webdriver_arguments,
            name=self.name
        )

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation

        raise NotImplementedError(
            f'WatchVimeoVideo is not implemented for architecture: {node.architecture}'
        )


class WatchVimeoVideoLinuxImplementation(Task):
    requirements = [
        "apt install -y python3-pip wget xvfb procps chromium chromium-driver",
        "pip3 install selenium webdriver-manager",
    ]

    def __init__(
        self,
        video_url: str,
        duration: Optional[int] = None,
        chrome_location: Optional[str] = None,
        webdriver_arguments: Optional[list] = None,
        *args,
        **kwargs
    ):
        self.video_url = video_url
        self.duration = duration
        self.chrome_location = chrome_location
        if not self.chrome_location:
            self.chrome_location = "/usr/bin/chromium"
        self.webdriver_arguments = webdriver_arguments
        super().__init__(*args, **kwargs)

    def run(self):
        return watch(self.video_url, self.duration, self.chrome_location, self.webdriver_arguments)


if __name__ == "__main__":
    print(watch("https://vimeo.com/440421754", 10))

In [21]:
class WatchStream(Task):
    def __init__(self, stream_url: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.stream_url = stream_url

    def run(self):
        # Simulate or fetch relevant data from the stream (e.g., bandwidth, latency)
        print(f"Monitoring stream at {self.stream_url}")
        # Placeholder for real stream monitoring (e.g., network stats, throughput, etc.)
        # Here you could log data, simulate fetching stream details, etc.
        return Success("Stream monitoring completed.")


In [22]:
class StartBackgroundWatcher(TaskDispatcher):
    def __init__(self, address: str, filepath: str, interval: int = 1, duration: int = 120, *args, **kwargs):
        """
        A watcher to monitor connectivity and log results to a file.
        
        :param address: Target IP or hostname to monitor.
        :param filepath: Path to log the watcher output.
        :param interval: Time between checks (in seconds).
        :param duration: Total duration of monitoring (in seconds).
        """
        super().__init__(*args, **kwargs)
        self.address = address
        self.filepath = filepath
        self.interval = interval
        self.duration = duration
        self.linux_implementation = StartBackgroundWatcherLinuxImplementation(
            address=self.address,
            filepath=self.filepath,
            interval=self.interval,
            duration=self.duration,
            *args,
            **kwargs
        )

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation
        raise NotImplementedError(
            f'StartBackgroundWatcher is not implemented for architecture: {node.architecture}'
        )


class StartBackgroundWatcherLinuxImplementation(Task):
    def __init__(self, address: str, filepath: str, interval: int = 1, duration: int = 120, *args, **kwargs):
        self.address = address.strip()
        self.filepath = filepath
        self.interval = interval
        self.duration = duration
        super().__init__(*args, **kwargs)

    def run(self):
        """
        Run the watcher process to monitor connectivity and log results.
        """
        try:
            with open(self.filepath, 'w') as outfile:
                start_time = time.time()
                while time.time() - start_time < self.duration:
                    # Run a single ping and capture its result
                    proc = subprocess.run(
                        ['ping', '-c', '1', self.address],  # Send a single ping
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE
                    )
                    timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
                    if proc.returncode == 0:
                        # Log success with response time
                        output = proc.stdout.decode('utf-8').strip()
                        outfile.write(f"{timestamp} SUCCESS: {output}\n")
                    else:
                        # Log failure with error details
                        error = proc.stderr.decode('utf-8').strip()
                        outfile.write(f"{timestamp} FAILURE: {error}\n")
                    
                    time.sleep(self.interval)
                
                return Success(f"Watcher ran for {self.duration} seconds.")
        except Exception as e:
            return Failure(f"Watcher encountered an error: {str(e)}")


In [23]:
class StopBackgroundPing(TaskDispatcher):
    def __init__(self, start_ping_task_name : str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.start_ping_task_name = start_ping_task_name
        self.linux_implementation = StopBackgroundPingLinuxImplementation(
            ping_task_name=self.start_ping_task_name,
            *args,
            **kwargs,
        )
    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation

        raise NotImplementedError(
            f'StopBackgroundPing is not implemented for {node.architecture}'
        )

class StopBackgroundPingLinuxImplementation(Task):
    requirements = ["sudo apt-get update", "sudo apt-get install -y tcpdump", "sudo apt-get install -y procps"]

    def __init__(self, ping_task_name: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ping_task_name = ping_task_name

    def run(self):
        signal.signal(signal.SIGCHLD, signal.SIG_IGN)
        pid = self.previous_steps.get(self.ping_task_name, [Failure("Named StartBackgroundPing not found")])[-1]
        if isinstance(pid, Failure):
            return pid

        pid = pid.unwrap()
        return subprocess.check_output(["kill", str(pid)])

In [24]:
class GetFileContents(TaskDispatcher):
    def __init__(self, filepath: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.linux_implementation = GetFileContentsLinuxImplementation(
            filepath=filepath, name=self.name
        )

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation
        raise NotImplementedError(
            f"GetFileContents is not implemented for architecture: {node.architecture}"
        )

class GetFileContentsLinuxImplementation(Task):
    def __init__(self, filepath: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filepath = filepath

    # TODO: implement the run() function
    def run(self):
        # 1. set the arguments for subprocess.run to exectue the command `cat self.filepath`
        args = ['cat', self.filepath]
        # 2. set the optional argument `capture_output` to True so that stdout will be captured
        result = subprocess.run(args, capture_output=True, text=False)
        
        if result.returncode != 0:
            return Failure(
                result.stdout.decode("utf-8").strip()
                + "\n"
                + result.stderr.decode("utf-8").strip()
            )

        return result.stdout.decode("utf-8")

In [8]:
pipeline = Pipeline()

# Flag to enable early stopping -- so if any task fails pipeline would go on working
# pipeline.early_stopping = False

# Generate data for Vimeo
# Generate data for Twitch
pipeline.then(

NameError: name 'StartCapture' is not defined

In [29]:
info = execute_pipeline(pipeline, working_node, 'background_ping_experiment')
display_results(info)

ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED
Node name: aws-fargate-A-cs190n2-1
Error: None
Result is: <class 'returns.result.Success'>
capture_task: [<Success: 8>]
d76f2f94-c665-47e2-8409-154558c7344d: [<Success: 5>]
background_ping: [<Success: 13>]
background_ping1: [<Success: 16>]
8a2271dc-00b7-42f1-aa7d-009760c1d6bb: [<Success: 30>]
93a7fea5-8aee-49c8-8b50-f9a8481d9610: [<Success: b''>]
a5f83e1b-e3db-4889-9860-b4545d79d31b: [<Success: b''>]
f04ebf04-2ec8-49f5-b25f-5eb42a61934e: [<Success: >]
22961f55-8f13-4f26-b8ef-9f9f26185099: [<Success: >]
04d43b78-b0d9-4424-9ca7-1489a980ab8e: [<Success: PING 8.8.8.8 (8.8.8.8) 56(84) bytes of data.
64 bytes from 8.8.8.8: icmp_seq=1 ttl=118 time=1.83 ms
64 bytes from 8.8.8.8: icmp_seq=2 ttl=118 time=1.29 ms
64 bytes from 8.8.8.8: icmp_seq=3 ttl=118 time=1.29 ms
64 bytes from 8.8.8.8: icmp_se